In [1]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Using TensorFlow backend.


 8192/57026 [===>..........................] - ETA: 3s

24576/57026 [===========>..................] - ETA: 1s

40960/57026 [====================>.........] - ETA: 0s

57344/57026 [==============================] - 1s 23us/step


In [4]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [6]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data[0].shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    
    return model



[-0.40342651  2.99178419 -1.33391162 -0.25683275 -1.21518188  1.89434613
 -1.91036058  1.24758524 -0.85646254 -0.34843254 -1.71818909  0.43190599
 -1.32920239]
